In [57]:
import os

def GetMoves(sgf):
    tmp=''
    for i,j in enumerate(sgf):
        if j=='(':
            tmp+=GetMoves(sgf[i:])
        elif j==')':
            tmp+=j
            return tmp
        else:
            tmp+=j
    return tmp

def ReadGame(game,path='./GoDat/KGS/'):
    g=open(path+game)
    return g.read()

def StripComments(game,path='/Users/nborggren/Codes/GamePDEs/Go/GoDat/KGS/'):
    os.system('perl CommentFilter.pl '+path+game+' '+path+game.replace('.sgf','_nocom.sgf'))
    
def GetKey(sgf,key='PW'):
    a=sgf.find(key+'[')
    b=sgf[a:].find(']')
    return sgf[a+len(key)+1:a+b]  

def StripKey(sgf,key='CR'):
    while sgf.find(key+'[')>0:
        tmp=GetKey(sgf,key=key)
        sgf=sgf.replace(key+'['+tmp+']','')
    return sgf

def CleanSGF(game,path='/Users/nborggren/Codes/GamePDEs/Go/GoDat/KGS/'):
    StripComments(game)
    sgf=ReadGame(game.replace('.sgf','_nocom.sgf'))
    for i in ['CR','WL','BL','OB','OW','C']:
        sgf=StripKey(sgf,key=i)
    g=open(path+game.replace('.sgf','_clean.sgf'),'w')
    g.write(sgf)
    

f=open('./GoDat/KGS/Borggren.games')
mygames=f.read().split('\n')
games = [i for i in mygames if i.find('.sgf')>0 and i.find('/Bogoliubon.sgf')<0]
myana = [i for i in mygames if i.find('/Bogoliubon.sgf')>0]
print len(games),len(mygames)
games.reverse()

5271 7229


In [5]:
games.reverse()

In [7]:
sgf=ReadGame(games[0])

In [8]:
print sgf

(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]
RU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]
PW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PC[The KGS Go Server at http://www.gokgs.com/]C[tf [1k\]: gg
Bogoliubon [1k\]: hello
]RE[B+Resign]
;B[pd]BL[594.971]C[Bogoliubon [1k\]: you too
]
;W[dp]WL[595.166]
;B[pp]BL[591.804]
;W[dc]WL[591.429]
;B[de]BL[583.597]
;W[ce]WL[578.434]
;B[dd]BL[576.991]
;W[cd]WL[575.075]
;B[ec]BL[576.342]
;W[cf]WL[558.182]
;B[db]BL[574.355]
;W[cc]WL[556.579]
;B[jc]BL[573.589]
;W[nq]WL[518.558]
;B[kp]BL[558.384]
;W[no]WL[473.748]
;B[pn]BL[547.169]
;W[pr]WL[471.679]
;B[qq]BL[541.444]
;W[kr]WL[469.681]
;B[qk]BL[540.433]
;W[qf]WL[463.26]
;B[qh]BL[529.959]
;W[qc]WL[458.875]
;B[pc]BL[526.683]
;W[qd]WL[456.531]
;B[pe]BL[525.706]
;W[rf]WL[454.596]
;B[og]BL[525]
;W[fc]WL[451.855]
;B[ed]BL[450.197]
;W[fb]WL[428.26]
;B[eb]BL[448.495]
;W[df]WL[423.273]
;B[fd]BL[445.608]
;W[ke]WL[418.305]
;B[ld]BL[432.425]
;W[pl]WL[415.033]
;B[ql]BL[430.263]
;W[cn]WL[372.689]
;B[fq]BL[407.454]
;W[e

In [31]:
for i in games:StripComments(i)

In [34]:
games[0]

'2016/2/24/Bogoliubon-tf.sgf'

In [58]:
CleanSGF(games[0])

In [59]:
sgf=ReadGame(games[0].replace('.sgf','_clean.sgf'))

In [60]:
sgf

'(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]\nRU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]\nPW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign]\n;B[pd]\n;W[dp]\n;B[pp]\n;W[dc]\n;B[de]\n;W[ce]\n;B[dd]\n;W[cd]\n;B[ec]\n;W[cf]\n;B[db]\n;W[cc]\n;B[jc]\n;W[nq]\n;B[kp]\n;W[no]\n;B[pn]\n;W[pr]\n;B[qq]\n;W[kr]\n;B[qk]\n;W[qf]\n;B[qh]\n;W[qc]\n;B[pc]\n;W[qd]\n;B[pe]\n;W[rf]\n;B[og]\n;W[fc]\n;B[ed]\n;W[fb]\n;B[eb]\n;W[df]\n;B[fd]\n;W[ke]\n;B[ld]\n;W[pl]\n;B[ql]\n;W[cn]\n;B[fq]\n;W[eq]\n;B[fp]\n;W[jq]\n;B[mp]\n;W[np]\n;B[lr]\n;W[lq]\n;B[kq]\n;W[mr]\n;B[jr]\n;W[ls]\n;B[iq]\n;W[dk]\n;B[em]\n;W[en]\n;B[fn]\n;W[fm]\n;B[el]\n;W[gm]\n;B[dn]\n;W[eo]\n;B[fo]\n;W[do]\n;B[cm]\n;W[dm]\n;B[dl]\n;W[cl]\n;B[ek]\n;W[bm]\n;B[dj]\n;W[ck]\n;B[ej]\n;W[im]\n;B[ij]\n;W[gj]\n;B[gk]\n;W[jp]\n;B[jo]\n;W[ip]\n;B[hp]\n;W[gn]\n;B[io]\n;W[hk]\n;B[hj]\n;W[ik]\n;B[gi]\n;W[ff]\n;B[gf]\n;W[jj]\n;B[ji]\n;W[ki]\n;B[kj]\n;W[jk]\n;B[jh]\n;W[kh]\n;B[jg]\n;W[kg]\n;B[jf]\n;W[kf]\n;B[lj]\n;W[mi]\n;B[mj]\n;W[ni]\n;B[lm]\n;W[ln]

In [28]:
sgf=StripKey(sgf,key='CR')

In [61]:
sgf.count(';')

200

In [62]:
sgf=sgf.replace('\n','')

In [68]:
moves=sgf.split(';')
moves=moves[2:]

In [78]:
locs=[i[2:4] for i in moves]
locs

['pd',
 'dp',
 'pp',
 'dc',
 'de',
 'ce',
 'dd',
 'cd',
 'ec',
 'cf',
 'db',
 'cc',
 'jc',
 'nq',
 'kp',
 'no',
 'pn',
 'pr',
 'qq',
 'kr',
 'qk',
 'qf',
 'qh',
 'qc',
 'pc',
 'qd',
 'pe',
 'rf',
 'og',
 'fc',
 'ed',
 'fb',
 'eb',
 'df',
 'fd',
 'ke',
 'ld',
 'pl',
 'ql',
 'cn',
 'fq',
 'eq',
 'fp',
 'jq',
 'mp',
 'np',
 'lr',
 'lq',
 'kq',
 'mr',
 'jr',
 'ls',
 'iq',
 'dk',
 'em',
 'en',
 'fn',
 'fm',
 'el',
 'gm',
 'dn',
 'eo',
 'fo',
 'do',
 'cm',
 'dm',
 'dl',
 'cl',
 'ek',
 'bm',
 'dj',
 'ck',
 'ej',
 'im',
 'ij',
 'gj',
 'gk',
 'jp',
 'jo',
 'ip',
 'hp',
 'gn',
 'io',
 'hk',
 'hj',
 'ik',
 'gi',
 'ff',
 'gf',
 'jj',
 'ji',
 'ki',
 'kj',
 'jk',
 'jh',
 'kh',
 'jg',
 'kg',
 'jf',
 'kf',
 'lj',
 'mi',
 'mj',
 'ni',
 'lm',
 'ln',
 'kn',
 'km',
 'kl',
 'll',
 'jm',
 'kk',
 'ml',
 'lk',
 'mk',
 'mm',
 'km',
 'nm',
 'ok',
 'nk',
 'nl',
 'pk',
 'oj',
 'je',
 'hf',
 'ph',
 'pj',
 'pg',
 'of',
 'oh',
 'pb',
 'qb',
 'qa',
 'ra',
 'pa',
 'cj',
 'ie',
 'qi',
 'rj',
 'qr',
 'rr',
 'rs',
 'fr',

In [90]:
letters='abcdefghijklmnopqrs'
bletters='srqponmlkjihgfedcba'
sym={}
for i,j in zip(letters,bletters):
    sym[i]=j

In [72]:
len(letters)

19

In [76]:
l,r=letters[0:10],letters[9:]

In [98]:
c1=[]
c2=[]
c3=[]
c4=[]
for i,j in zip(moves,locs):
    if j[0] in l and j[1] in l:
        c1.append(i)
    if j[0] in l and j[1] in r:
        tmp=i.replace(j[1],sym[j[1]])
        c2.append(tmp)
    if j[0] in r and j[1] in r:
        tmp=i.replace(j[1],sym[j[1]])
        tmp=tmp.replace(j[0],sym[j[0]])
        c3.append(tmp)
    if j[0] in r and j[1] in l:
        tmp=i.replace(j[0],sym[j[0]])
        c4.append(tmp)


In [102]:
c1

['W[dc]',
 'B[de]',
 'W[ce]',
 'B[dd]',
 'W[cd]',
 'B[ec]',
 'W[cf]',
 'B[db]',
 'W[cc]',
 'B[jc]',
 'W[fc]',
 'B[ed]',
 'W[fb]',
 'B[eb]',
 'W[df]',
 'B[fd]',
 'B[dj]',
 'B[ej]',
 'B[ij]',
 'W[gj]',
 'B[hj]',
 'B[gi]',
 'W[ff]',
 'B[gf]',
 'W[jj]',
 'B[ji]',
 'B[jh]',
 'B[jg]',
 'B[jf]',
 'W[je]',
 'B[hf]',
 'W[cj]',
 'B[ie]',
 'B[ci]',
 'W[bi]',
 'B[ch]',
 'W[bh]',
 'B[cb]',
 'W[jd]',
 'B[ic]',
 'B[fg]',
 'W[eg]',
 'B[eh]',
 'W[hi]',
 'B[ii]',
 'W[cg]',
 'B[dh])']

In [86]:
loc1

['W[dp]',
 'W[cn]',
 'B[fq]',
 'W[eq]',
 'B[fp]',
 'W[jq]',
 'B[jr]',
 'B[iq]',
 'W[dk]',
 'B[em]',
 'W[en]',
 'B[fn]',
 'W[fm]',
 'B[el]',
 'W[gm]',
 'B[dn]',
 'W[eo]',
 'B[fo]',
 'W[do]',
 'B[cm]',
 'W[dm]',
 'B[dl]',
 'W[cl]',
 'B[ek]',
 'W[bm]',
 'B[dj]',
 'W[ck]',
 'B[ej]',
 'W[im]',
 'B[ij]',
 'W[gj]',
 'B[gk]',
 'W[jp]',
 'B[jo]',
 'W[ip]',
 'B[hp]',
 'W[gn]',
 'B[io]',
 'W[hk]',
 'B[hj]',
 'W[ik]',
 'W[jj]',
 'W[jk]',
 'B[jm]',
 'W[cj]',
 'B[fr]',
 'B[dr]',
 'W[cq]',
 'B[cr]',
 'W[br]',
 'B[bs]',
 'W[ar]']

In [105]:
sgf

'(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]RU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]PW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign];B[pd];W[dp];B[pp];W[dc];B[de];W[ce];B[dd];W[cd];B[ec];W[cf];B[db];W[cc];B[jc];W[nq];B[kp];W[no];B[pn];W[pr];B[qq];W[kr];B[qk];W[qf];B[qh];W[qc];B[pc];W[qd];B[pe];W[rf];B[og];W[fc];B[ed];W[fb];B[eb];W[df];B[fd];W[ke];B[ld];W[pl];B[ql];W[cn];B[fq];W[eq];B[fp];W[jq];B[mp];W[np];B[lr];W[lq];B[kq];W[mr];B[jr];W[ls];B[iq];W[dk];B[em];W[en];B[fn];W[fm];B[el];W[gm];B[dn];W[eo];B[fo];W[do];B[cm];W[dm];B[dl];W[cl];B[ek];W[bm];B[dj];W[ck];B[ej];W[im];B[ij];W[gj];B[gk];W[jp];B[jo];W[ip];B[hp];W[gn];B[io];W[hk];B[hj];W[ik];B[gi];W[ff];B[gf];W[jj];B[ji];W[ki];B[kj];W[jk];B[jh];W[kh];B[jg];W[kg];B[jf];W[kf];B[lj];W[mi];B[mj];W[ni];B[lm];W[ln];B[kn];W[km];B[kl];W[ll];B[jm];W[kk];B[ml];W[lk];B[mk];W[mm];B[km];W[nm];B[ok];W[nk];B[nl];W[pk];B[oj];W[je];B[hf];W[ph];B[pj];W[pg];B[of];W[oh];B[pb];W[qb];B[qa];W[ra];B[pa];W[cj];B[ie];W[qi];B[rj];W[qr];B[rr];W[rs]

In [91]:
sym['s']

'a'

In [103]:
header="(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]\nRU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]\nPW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign]\n;"

In [104]:
header

'(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]\nRU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]\nPW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign]\n;'

In [112]:
f=open('test.sgf','w')
record=header
for i in c1[:-1]:record+=i+';'
record+=c1[-1].replace(')','')+')'

record+='\n'+header
for i in c2[:-1]:record+=i+';'
record+=c2[-1].replace(')','')+')'

record+='\n'+header
for i in c1[:-1]:record+=i+';'
record+=c1[-1].replace(')','')+')'

record+='\n'+header
for i in c1[:-1]:record+=i+';'
record+=c1[-1].replace(')','')+')'

f.write(record)

In [114]:
print record

(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]
RU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]
PW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign]
;W[dc];B[de];W[ce];B[dd];W[cd];B[ec];W[cf];B[db];W[cc];B[jc];W[fc];B[ed];W[fb];B[eb];W[df];B[fd];B[dj];B[ej];B[ij];W[gj];B[hj];B[gi];W[ff];B[gf];W[jj];B[ji];B[jh];B[jg];B[jf];W[je];B[hf];W[cj];B[ie];B[ci];W[bi];B[ch];W[bh];B[cb];W[jd];B[ic];B[fg];W[eg];B[eh];W[hi];B[ii];W[cg];B[dh])
(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]
RU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]
PW[Bogoliubon]PB[tf]WR[1k]BR[1k]DT[2016-02-24]PRE[B+Resign]
;W[dd];W[cf];B[fc];W[ec];B[fd];W[jc];B[jb];B[ic];W[di];B[eg];W[ef];B[ff];W[fg];B[eh];W[gg];B[df];W[ee];B[fe];W[de];B[cg];W[dg];B[dh];W[ch];B[ei];W[bg];B[dj];W[ci];B[ej];W[ig];B[ij];W[gj];B[gi];W[jd];B[je];W[id];B[hd];W[gf];B[ie];W[hi];B[hj];W[ii];W[jj];W[ji];B[jg];W[cj];B[fb];B[db];W[cc];B[cb];W[bb];B[ba];W[ab])
(;GM[1]FF[4]CA[UTF-8]AP[CGoban:3]ST[2]
RU[Japanese]SZ[19]KM[6.50]TM[600]OT[5x40 byo-yomi]
PW[Bogoliubo

In [111]:
c1

['W[dc]',
 'B[de]',
 'W[ce]',
 'B[dd]',
 'W[cd]',
 'B[ec]',
 'W[cf]',
 'B[db]',
 'W[cc]',
 'B[jc]',
 'W[fc]',
 'B[ed]',
 'W[fb]',
 'B[eb]',
 'W[df]',
 'B[fd]',
 'B[dj]',
 'B[ej]',
 'B[ij]',
 'W[gj]',
 'B[hj]',
 'B[gi]',
 'W[ff]',
 'B[gf]',
 'W[jj]',
 'B[ji]',
 'B[jh]',
 'B[jg]',
 'B[jf]',
 'W[je]',
 'B[hf]',
 'W[cj]',
 'B[ie]',
 'B[ci]',
 'W[bi]',
 'B[ch]',
 'W[bh]',
 'B[cb]',
 'W[jd]',
 'B[ic]',
 'B[fg]',
 'W[eg]',
 'B[eh]',
 'W[hi]',
 'B[ii]',
 'W[cg]',
 'B[dh])']